# Running an agent in one file

The standard way to run an agent in PySC2 is simple. However, it's built on top of a few abstractions that make it difficult to know how modify the source code (e.g. training our own reinforcement learning agent). This purpose of this notebook is to elucidate the PySC2 pipeline for initializing an agent and having it interact with the environment. Most of the code here is adapted from the PySC2 source code but has been modified to work end-to-end in one file. 

In [1]:
'''Imports and resolving issues to make notebook work'''
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import json
import numpy
import importlib
import threading
import time
from future.builtins import range

# Issue #1 that I ran into and solution: https://github.com/chris-chris/pysc2-examples/issues/5#issuecomment-342088938

# Issue #2: Apparently jupyter notebook sometimes passes some command line arguments. The -f flag confuses the pysc2
# library which will result in an error. Removing that argument is a hacky way to get through
# this. 
print(sys.argv)
sys.argv[1:] = []

from pysc2 import maps
from pysc2.env import sc2_env
from pysc2.env import available_actions_printer
from pysc2.lib import actions
from pysc2.lib import features
from pysc2.lib import stopwatch
from pysc2.bin import agent
from pysc2.agents import base_agent

['/Users/asoong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py', '-f', '/Users/asoong/Library/Jupyter/runtime/kernel-909caaf1-270e-493f-ae64-fef7ef45ac21.json']


In [2]:
'''A simple random agent that shows how to select from available function ids and type ids'''
class RandomAgent(base_agent.BaseAgent):
    """A random agent for starcraft."""

    def step(self, obs):
        super(RandomAgent, self).step(obs)
        function_id = numpy.random.choice(obs.observation.available_actions)
        args = [[numpy.random.randint(0, size) for size in arg.sizes]
                for arg in self.action_spec.functions[function_id].args]
        return actions.FunctionCall(function_id, args)

In [3]:
'''Some arguments that would have been passed from command line if agent was ran normally'''
FLAG_map = "MoveToBeacon"
FLAG_agent = RandomAgent
FLAG_agent_race = "random"
FLAG_profile = False
FLAG_trace = False
FLAG_parallel = 1 # How many instances to run in parallel.
FLAG_feature_screen_size = 84
FLAG_feature_minimap_size = 64
FLAG_rgb_screen_size = None
FLAG_rgb_minimap_size = None
FLAG_action_space = None
FLAG_use_feature_units = False
FLAG_step_mul = 8
FLAG_game_steps_per_episode = None
FLAG_render = True # Whether to render with pygame.
FLAG_max_agent_steps = 0
FLAG_max_episodes = 4
FLAG_save_replay = False

'''Some other settings'''
map_inst = maps.get(FLAG_map)

agent_classes = []
players = []

agent_cls = FLAG_agent
agent_classes.append(agent_cls)
players.append(sc2_env.Agent(sc2_env.Race[FLAG_agent_race]))

print(map_inst)
print(agent_classes)
print(players)

stopwatch.sw.enabled = FLAG_profile or FLAG_trace
stopwatch.sw.trace = FLAG_trace

threads = []
# currently only one thread will run
for _ in range(FLAG_parallel - 1):
    print("Multiple threads being used")
    t = threading.Thread(target=run_thread,
                         args=(agent_classes, players, FLAGS.map, False))
    threads.append(t)
    t.start()

MoveToBeacon
    mini_games/MoveToBeacon.SC2Map
    players: 1, score_index: 0, score_multiplier: 1
    step_mul: 8, game_steps_per_episode: 0
[<class '__main__.RandomAgent'>]
[Agent(race=<Race.random: 4>)]


In [4]:
'''The main loop where agents and environment interacts'''
def run_loop(agents, env, max_frames=0, max_episodes=0):
    total_frames = 0
    total_episodes = 0
    start_time = time.time()

    observation_spec = env.observation_spec()
    action_spec = env.action_spec()
    for agent, obs_spec, act_spec in zip(agents, observation_spec, action_spec):
        agent.setup(obs_spec, act_spec)

    try:
        while not max_episodes or total_episodes < max_episodes:
            total_episodes += 1
            timesteps = env.reset()
            for a in agents:
                a.reset()
            while True:
                total_frames += 1
                actions = [agent.step(timestep)
                            for agent, timestep in zip(agents, timesteps)]
                if max_frames and total_frames >= max_frames:
                    return 
                if timesteps[0].last():
                    break
                timesteps = env.step(actions)
    except KeyboardInterrupt:
        pass
    finally:
        elapsed_time = time.time() - start_time
        print("Took %.3f seconds for %s steps: %.3f fps" % (
            elapsed_time, total_frames, total_frames / elapsed_time))

In [7]:
'''
Create an AgentInterfaceFormat to pass into the environment initializer.  
Must do this in the newest version of pysc2
'''
FLAG_agent_interface_format = features.parse_agent_interface_format(
    feature_screen=FLAG_feature_screen_size,
    feature_minimap=FLAG_feature_minimap_size,
    rgb_screen=FLAG_rgb_screen_size,
    rgb_minimap=FLAG_rgb_minimap_size,
    action_space=FLAG_action_space,
    camera_width_world_units=64,
    use_feature_units=FLAG_use_feature_units,
)

In [8]:
"""Run one thread worth of the environment with agents."""
with sc2_env.SC2Env(
    map_name=FLAG_map,
    players=players,
#     feature_screen_size=FLAG_feature_screen_size,
#     feature_minimap_size=FLAG_feature_minimap_size,
#     rgb_screen_size=FLAG_rgb_screen_size,
#     rgb_minimap_size=FLAG_rgb_minimap_size,
#     action_space=FLAG_action_space,
#     use_feature_units=FLAG_use_feature_units,
    agent_interface_format=FLAG_agent_interface_format,
    step_mul=FLAG_step_mul,
    game_steps_per_episode=FLAG_game_steps_per_episode,
    visualize=FLAG_render) as env:
        env = available_actions_printer.AvailableActionsPrinter(env)
        agents = [agent_cls() for agent_cls in agent_classes]     
        run_loop(agents, env, FLAG_max_agent_steps, FLAG_max_episodes)
        if FLAG_save_replay:
            env.save_replay(agent_classes[0].__name__)

for t in threads:
    t.join()

if FLAG_profile:
    print(stopwatch.sw)

UnparsedFlagAccessError: Trying to access flag --sc2_run_config before flags were parsed.